In [318]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series, DataFrame
% matplotlib inline
from itertools import combinations
import re
import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [319]:
df = pd.read_csv('top_100_50_years.csv')

In [320]:
df.head()

,Unnamed: 0,Arists,Date,Entry,Peak,Song,Weeks
0,359,PANIC! AT THE DISCO,4/7/2018,60,60,Say Amen (Saturday Night),2
1,363,SHAWN MENDES,4/7/2018,64,64,Lost In Japan,1
2,369,FAMOUS DEX,4/7/2018,70,29,Japan,3
3,385,MORGAN WALLEN featuring FLORIDA GEORGIA LINE,4/7/2018,86,83,Up Down,3
4,386,KEALA SETTLE & THE GREATEST SHOWMAN ENSEMBLE,4/7/2018,87,87,This Is Me,1


In [321]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62579 entries, 0 to 62578
Data columns (total 7 columns):
Unnamed: 0    62579 non-null int64
Arists        62579 non-null object
Date          62579 non-null object
Entry         62579 non-null int64
Peak          62579 non-null int64
Song          62579 non-null object
Weeks         62579 non-null int64
dtypes: int64(4), object(3)
memory usage: 3.3+ MB


In [322]:
all_artists = df.Arists

In [323]:
# split on ", " " and " " AND " " & " " featuring " " FEATURING " " x " " X "
def separate_artists(all_artists):
    ret_list = []
    for colab in all_artists:
        colabs = re.split(' and |, | AND | & | featuring | FEATURING | x | X | feat ',colab)
        ret_list.append(colabs)
    return ret_list
    
all_artists_test = ['PANIC! AT THE DISCO',
 'SHAWN MENDES',
 'FAMOUS DEX',
 'MORGAN WALLEN featuring FLORIDA GEORGIA LINE',
 'KEALA SETTLE & THE GREATEST SHOWMAN ENSEMBLE']

print separate_artists(all_artists_test)

[['PANIC! AT THE DISCO'], ['SHAWN MENDES'], ['FAMOUS DEX'], ['MORGAN WALLEN', 'FLORIDA GEORGIA LINE'], ['KEALA SETTLE', 'THE GREATEST SHOWMAN ENSEMBLE']]


In [324]:
lst = [1, 2, 3, 4, 5]
def song_artist_combi(lst):
    colabs = []
    if len(lst) != 1:
        for combo in combinations(lst, 2):  
            colabs.append(combo)
    else:
        return lst
    return colabs
    
print song_artist_combi(lst)

[(1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


In [325]:
artists_separated = separate_artists(all_artists)
#print artists_separated

In [326]:
edges_list = []
for colab in artists_separated:
    edges_list.append(song_artist_combi(colab))
edges_tuples = [item for sublist in edges_list for item in sublist if type(item) == tuple]
#for edge in edges_tuples:
    #print edge

In [327]:
len(edges_tuples)

14911

In [328]:
edges_set = set(edges_tuples)
len(edges_set)

3419

In [329]:
edges = list(edges_set)
# the set of edges has not repeat colabs
#for colab in edges:
    #print colab

In [330]:
nodes_set = set()
for colab in edges:
    for artist in colab:
        nodes_set.add(artist)
nodes = list(nodes_set)
len(nodes)

2528

In [331]:
# print all artists
#for artist in nodes:
    #print artist

In [302]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [317]:
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.models import Circle, HoverTool, MultiLine, PanTool, WheelZoomTool
from bokeh.models.graphs import from_networkx
from bokeh.models import Range1d, Plot

plot_options = dict(tools='pan,wheel_zoom')

# We could use figure here but don't want all the axes and titles
plot = Plot(x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

# Create a Bokeh graph from the NetworkX input using nx.spring_layout
graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

# Blue circles for nodes, and light grey lines for edges
graph.node_renderer.glyph = Circle(size=10, fill_color='#2b83ba')
graph.edge_renderer.glyph = MultiLine(line_color="#cccccc", line_alpha=0.8, line_width=2)

# green hover for both nodes and edges
graph.node_renderer.hover_glyph = Circle(size=15, fill_color='#abdda4')
graph.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=4)

# When we hover over nodes, highlight adjecent edges too
graph.inspection_policy = NodesAndLinkedEdges()

plot.add_tools(HoverTool(tooltips=None), PanTool(), WheelZoomTool())

show(plot)